In [87]:
import scipy.stats as sps
import numpy as np
import numpy_financial as npf
import pandas as pd

# Question 1

> 1. An individual divides an investment between hedge funds that earn (before fees) -21%, -11%, +21%, +25%, +27%, and +31%.  All hedge funds charge 2 plus 20%. 
>
> - What is the overall return on the investments? 
>
> - How is it divided between the hedge fund and the investor? 
> 
> - How does your answer change if a fund of funds charging 1 plus 5% is used. 
>
> (Assume that a hedge fund’s incentive fee of 20% is paid on profits net of the management fee. The fund of fund incentive fee of 5% applies to the total profits, net of management fees, from the hedge funds.)

Assuming that this individual is dividing his/her investment to the listed hedge funds in equal portions, the breakdown of the overall investment return is as followed: 

- Overall return, before fee: **12%**, which can be divided into: 
  - Fee payment of fund managers: **6.8%**
  - Profit retained by the investor: **5.2%**
  
If a fund of fund rate of 1 plus 5% is further charged, then the breakdown will be changed: 

- Overall return before fee will remain unchanged: **12%**, including: 
  - Fee payment of fund managers: **5.51%**
  - (New) fee payment to investment managers (fund of funds): **1.29%**
  - Profit retained by the investor: **5.2%**

Detailed calculation can be seen below: 

In [85]:
def fee_calculation(pre_fee_profit: float, management_fee: float = 0.02, incentive_rate: float = 0.2, verbose = 1): 
    if pre_fee_profit <= 0: 
        # For un-profited funds, only management fee applies: 
        man_fee = management_fee
        inc_fee = 0
    else: 
        # FOr profit funds: 
        man_fee = management_fee
        inc_fee = incentive_rate * (pre_fee_profit - management_fee)
    total_fee = man_fee + inc_fee
    total_profit = pre_fee_profit - total_fee
    if verbose ==1: 
        print(f'Fund return (before fees): {100*pre_fee_profit}%, fee at {100*management_fee} plus {100*incentive_rate}% | Total fee: {100*total_fee:.1f}% | Post-fee return {100*total_profit:.1f}%')
    return (total_fee, total_profit)

In [86]:
pre_fee_profits = [-0.21, -0.11, 0.21, 0.25, 0.27, 0.31]
fee = []
overall_return = []

for profit in pre_fee_profits: 
    result = fee_calculation(profit)
    fee.append(result[0])
    overall_return.append(result[1])

fund_of_funds = fee_calculation(np.mean(overall_return), 0.01, 0.05, 0)

print('---------------------------------------------------------')
print(f'overall fund return, before fees: {np.mean(pre_fee_profits)*100:.2f}%')
print(f'overall fund return, after fees: {np.mean(overall_return)*100:.2f}%')
print(f'dividend paid to individual fund managers: {np.mean(fee)*100:.2f}%')
print('---------------------------------------------------------')
print(f'overall fund return, after hedge fund fees & fund of funds fee: {np.mean(fund_of_funds[1])*100:.2f}%')
print(f'dividend paid to individual fund managers: {np.mean(fee)*100:.2f}%')
print(f'dividend paid to investment manager: {np.mean(fund_of_funds[0])*100:.2f}%')

Fund return (before fees): -21.0%, fee at 2.0 plus 20.0% | Total fee: 2.0% | Post-fee return -23.0%
Fund return (before fees): -11.0%, fee at 2.0 plus 20.0% | Total fee: 2.0% | Post-fee return -13.0%
Fund return (before fees): 21.0%, fee at 2.0 plus 20.0% | Total fee: 5.8% | Post-fee return 15.2%
Fund return (before fees): 25.0%, fee at 2.0 plus 20.0% | Total fee: 6.6% | Post-fee return 18.4%
Fund return (before fees): 27.0%, fee at 2.0 plus 20.0% | Total fee: 7.0% | Post-fee return 20.0%
Fund return (before fees): 31.0%, fee at 2.0 plus 20.0% | Total fee: 7.8% | Post-fee return 23.2%
---------------------------------------------------------
overall fund return, before fees: 12.00%
overall fund return, after fees: 6.80%
dividend paid to individual fund managers: 5.20%
---------------------------------------------------------
overall fund return, after hedge fund fees & fund of funds fee: 5.51%
dividend paid to individual fund managers: 5.20%
dividend paid to investment manager: 1.29%


\pagebreak
# Question 2

> 2. How does Table 7.1 in text change if the principal assigned to the senior, mezzanine, and equity tranche in Figure 7.4 are 72%, 22%, and 6% for the ABS and 70%, 25% and 5% for the ABS CDO?

In [157]:
abs_combo = pd.Series({'senior': 0.72, 'mezzanine': 0.22, 'equity': 0.06}) #AAA, BBB, Equity
cdo_combo = pd.Series({'senior': 0.70, 'mezzanine': 0.25, 'equity': 0.05}) #AAA, BBB, Equity

losses = [0.1, 0.15, 0.2, 0.25]
sub_losses = []
losses_to_tranches = pd.DataFrame()

for loss in losses: 
    # Calculate losses to subprime portfolio: 
    mez_trench_loss = (loss - abs_combo.equity)/abs_combo.mezzanine
    sub_losses.append(mez_trench_loss)
    # Calculate losses to ABS CDO: 
    cdo_loss = pd.Series(np.zeros(3), index=['equity', 'mezzanine', 'senior'])
    loss = mez_trench_loss
    for tranch in ['equity', 'mezzanine', 'senior']: 
        if loss >= cdo_combo[tranch]: 
            cdo_loss[tranch] = 1
            loss -= cdo_combo[tranch]
        else: 
            cdo_loss[tranch] = loss/cdo_combo[tranch]
    losses_to_tranches[mez_trench_loss] = cdo_loss

losses_to_tranches = losses_to_tranches.T.reset_index()
losses_to_tranches['Losses to Subprime Portfolios'] = losses
losses_to_tranches = losses_to_tranches.iloc[:, [4,0,1,2,3]]
losses_to_tranches.rename(columns = {'index': 'Losses to Mezzanine Tranche of ABS',
                                     'equity': 'Losses to Equity Tranche of ABS CDO',
                                     'mezzanine': 'Losses to Mezzanine Tranche of ABS CDO',
                                     'senior': 'Losses to Senior Tranche of ABS CDO'}, inplace=True)

def change_data_type(cell): 
    return format(cell, ".0%")
losses_to_tranches = losses_to_tranches.map(change_data_type)
losses_to_tranches




,Losses to Subprime Portfolios,Losses to Mezzanine Tranche of ABS,Losses to Equity Tranche of ABS CDO,Losses to Mezzanine Tranche of ABS CDO,Losses to Senior Tranche of ABS CDO
0,10%,18%,100%,53%,19%
1,15%,41%,100%,100%,16%
2,20%,64%,100%,100%,48%
3,25%,86%,100%,100%,81%


\pagebreak
# Question 3

> 3.	Variable x has a uniform distribution with values between 5 and 15 being equally likely. Variable y has a Pareto distribution. A Gaussian copula is used to define the correlation between the two distributions. The Pareto distribution for variable y has a probability density function:
$$
\frac{ac^{a}}{y^{a+1}}
$$
> For value of $y$ between $c$ and $infinity$ where $c=4$ and $a=0.5$. Produce a table similar to Table 9.5 in the text considering values of $x$ equal to 7, 9, 11, and 13 and values of $y$ equal to 5, 10, 30 and 60. Assume a copula correlation of 0.4. A spreadsheet for calculating the cumulative bivariate normal distribution is provided on the author's website `www-2.rotman.utoronto.ca/~hull/riskman`. 


Mapping of given values to standard normal distribution is done below. Further calculation of cumulative joint probability distribution of both values under bivirate normal distribution with a copula correlation of 0.4 is done in the given spreadsheet, with the result attached below. 

In [210]:
x = [7, 9, 11, 13]
y = [5, 10, 30, 50]

x_dist = sps.uniform(loc=5, scale=15-5)
y_dist = sps.pareto(b=0.5, scale = 4) #scale = c
std_norm = sps.norm

In [213]:
x_percentile = [x_dist.cdf(x_val) for x_val in x]
x_map = [std_norm.ppf(x_perc) for x_perc in x_percentile]

y_percentile = [y_dist.cdf(y_val) for y_val in y]
y_map = [std_norm.ppf(y_perc) for y_perc in y_percentile]

print(f'Percentile of distribution of x values: {x_percentile}')
print(f'Mapped x values to std. normal dist.:   {x_map}')
print(f'Percentile of distribution of y values: {y_percentile}')
print(f'Mapped y values to std. normal dist.:   {y_map}')

Percentile of distribution of x values: [0.2, 0.4, 0.6, 0.8]
Mapped x values to std. normal dist.:   [-0.8416212335729142, -0.2533471031357997, 0.2533471031357997, 0.8416212335729143]
Percentile of distribution of y values: [0.10557280900008414, 0.3675444679663241, 0.6348516283298893, 0.717157287525381]
Mapped y values to std. normal dist.:   [-1.2504214910006977, -0.33836395189680224, 0.34473082330391736, 0.5744173352351937]


\pagebreak
# Question 4

> 4. Five years of history for the S&P 500 is attached. March 2020 was a volatile period for the index. Imagine that it is March 13, 2020. Use the previous 251 days (250 percentage changes) to calculate the one-day value at risk and expected shortfall for a portfolio with $1000 invested in the index. Ignore dividends. Provide results for four different methods: 
>
> - a) The basic historical simulation approach
>
> - b) Exponential weighting with $\lambda = 0.995$
>
> - c) Volatility scaling with $\lambda=0.94$ (assume an initial variance equal to the sample variance for the 250 changes)
>
> - d) Extreme value theory with $u=25$

In [4]:
def pvg(pmt, i, g, t): 
    '''
    Function to calculate the present value of a growing annuity (PVG), where: 
    pmt - annuity paid per term
    i - interest rate per term
    g - growth rate of payment per period
    t - number of period
    '''
    return pmt / (i - g) * (1 - ((1 + g) / (1 + i))**t)

def gfv(fv,i, g, t): 
    '''
    Function to calculate the growing annuity required to fulfill a certain future value, 
    where the growth rate of such annuity is known, where: 
    fv - future value target
    i - interest rate per term
    g - growth rate of payment per period
    t - number of period
    '''
    return fv * (i - g) / ((1 + i)**t - (1 + g)**t)

\pagebreak
Actual *calculations of Q4 can be referred here:*

In [5]:
# Assumptions
start_income = 1 # In real dollars
print(f'first year income: {start_income:.4f}')
# Given parameters: 
real_rate = 0.02
work_length = 45
pension_payout = 0.7
pension_rate = -0.01
pension_length = 18
bond_return = 0.015

# Calculation: 
# Employee's salary at his final year: 
# FV(PV=income, i=2%, t=44): - Note that the salary @ EOY1 is 1 
final_income = npf.fv(real_rate, work_length-1, pmt=0, pv=-start_income)
print(f'final_income: {final_income:.4f}')

# Annual pension payout at the starting year: income * 70%
annual_pension = final_income * pension_payout
print(f'starting annual pension: {annual_pension:.4f}')

# Total pension required: pension payout increased by pension rate and compounded by bond return
# PV(A=annual_pension, i=1.5%, g=-1%, t=18)
total_pension = pvg(pmt=annual_pension, i=bond_return, g=pension_rate, t=pension_length)
print(f'total pension payment: {total_pension:.4f}')

# Note: First pension payout should be at the beginning of the starting year,
# .... which is essentially the end of the final year: same term of final salary

# Percentage: While working, the compensation increased as salary increase and compounded by bond return
# A(FV=total_pension, i=1.5%, g=2%, t=45)
annual_contribution = gfv(fv=total_pension, i=bond_return, g=real_rate, t=work_length)
print(f'annual contribution required: {annual_contribution:.4f}')

first year income: 1.0000
final_income: 2.3901
starting annual pension: 1.6730
total pension payment: 24.2035
annual contribution required: 0.2502
